### Project: INM713-coursework
**Author**: Zac Detorakis (zacharias.detorakis@city.ac.uk)

**Version**: 1.0<br>
**Date**: 15/03/2021

In [22]:
# Import Libraries
import sys


sys.path.append('./lib/')


from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
import pandas as pd
import math
from SPARQLWrapper import SPARQLWrapper, JSON
from stringcmp import isub
from lookup import DBpediaLookup
import yake
import re

import owlrl


# Instanciate the class

In [2]:
class FinalCoursework(object):
    """
    This will contain the solution for the coursework for the SW&KGT
    """
    
    def __init__(self, input_csv):
        
        # The file containing the data to load in the KG
        self.file = input_csv
        
        #Intialise the KG
        self.g = Graph()
        
        #setup the ontology IRI used for the courseworkontology..
        self.zdetor_ns_str= "https://www.city.ac.uk/ds/inm713/zacharias_detorakis#"
        #Special namspaces class to create directly URIRefs in python.           
        self.zdetor = Namespace(self.zdetor_ns_str)
        #Prefixes for the serialization
        self.g.bind("zdetor", self.zdetor)
        
        self.classStringToURI = dict()
        
        
        #read the raw data into a dataframe
#         self.df = pd.read_csv(filepath_or_buffer = self.file, sep=',', quotechar='"',escapechar="\\")
        self.df = []
    def is_nan(self, x):
        return (x != x)

if __name__ == '__main__':
    input_csv = "./input_files/INM713_coursework_data_pizza_8358_1_reduced.csv"
    input_csv = "./input_files/INM713_coursework_data_pizza_8358_1_reduced - small.csv"
    
    solution = FinalCoursework(input_csv)
    solution.df = pd.read_csv(filepath_or_buffer = solution.file, sep=',', quotechar='"',escapechar="\\")
    
        

# 2. DATA PRE-PROCESSING
## 2.1 Restaurants

First we inspect the restaurants and we see that there are more addresses than restaurants. Based on our ontology a restaurant can only be in one address therefore we need to idenditify those restaurant and create a new URI for them.

In [3]:
print(f"Number of unique restaurant names: {len(solution.df.groupby(['name']))}")
print(f"Number of unique address lines: {len(solution.df.groupby(['name','address']))}")

Number of unique restaurant names: 10
Number of unique address lines: 13


In [4]:
def findNewRestaurantName(original_value,mapping_dict):
    """
    A function used to map a given value to a new one as defined in the mapping dictionary
    ...

    Attributes
    ----------
    original_value : str
        the original value to be mapped. In this case this will be the concatenated field (name+address)
    mapping_dict : dict
        this is a dictionary where the 'key' is the original value and the 'value' is the new value it maps to
    """
    try:
        return mapping_dict[original_value]
    except:
        return 'invalid'

def createNewRestaurantNames(original_df):
    """
    A function used to create a new name for restaurants that share the same name but based on the addresses seem to be different. The new name will be created as a new 'restaurant_name' column in the df 
    ...

    Attributes
    ----------
    original_df : dataframe
        the original dataframe to be updated with the new_name column
    """
    
    # group the restaurant by name and filter out any restaurant with exactly one address
    df_group = pd.DataFrame(original_df.groupby('name')['address'].nunique())
    df_dup_restaurants = df_group[df_group.address>1]

    # next we create a new version of the df to add the new name column
    new_df = solution.df
    # for now populate the column with a concatenation of name and address
    new_df['restaurant_name']=new_df.apply(lambda x:'%s_%s' % (x['name'],x['address']),axis=1)

    # next we create a termporary dataframe to store the name and concatenated column. drop the duplicates (from the multiple menu items) and sort the df
    temp = new_df[['name', 'restaurant_name']].drop_duplicates()
    temp.sort_values(by='name',inplace=True)

    # finally create a dictionary mappin the concatenated field to a new restaurant name by 
    # - adding a sequence number at the end of the duplicates or 
    # - reusing the existing name for restaurants that appear only once
    prev_name = ''
    incr = 1
    new_name_dict = {}
    for index, row in temp.iterrows():
        if row['name'] == prev_name:
            incr += 1
        else:
            incr = 1

        if row['name'] in str(df_dup_restaurants.index):
            new_name_dict[row['restaurant_name']]= row['name'] + '___' + str(incr)
            prev_name = row['name']
        else:
            new_name_dict[row['restaurant_name']]= row['name']

    # Finally apply the function to map the concatenated field to the new name
    new_df['restaurant_name'] = new_df['restaurant_name'].apply(lambda x: findNewRestaurantName(x,new_name_dict))
    # new_df.to_csv("temp.csv")
    return new_df

In [5]:
solution.df = createNewRestaurantNames(solution.df)
solution.df.head()

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,restaurant_name
0,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Any Brick Oven Pizza,NaN,NaN,Small,Bertucci's___1
1,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Bbq Chicken Pizza,10.99,USD,BBQ chicken marinated in a smoky hickory BBQ s...,Bertucci's___1
2,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Buffalo Chicken Flatbread Pizza,8.49,USD,Grilled chicken tossed with buffalo sauce. Top...,Bertucci's___1
3,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Cheese Party Pizza,5.00,USD,NaN,Bertucci's___1
4,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Cheese Pizza,10.29,USD,NaN,Bertucci's___1


## 2.2 States

First of all we need to perform some exploratory data analysis before we start loading the data to ensure the data consistency and perhaps perform a bit of data cleansing.
We will start with the address part as that is the easiest one to profile.

Given that the addresses are unique per restaurant and then repeated for all menu items of that restaurant we will get a subsection of the df to better assess the extent of problematic records that need fixing. As per our ontology, the address properties are:
* address (i.e. representing the address line)
* city
* country
* state and
* postcode

In [6]:
def convertPostCodeStringToPostCodes(post_code):

    #create a new empty list for the post codes
    post_code_list = []
    
    #separate the post codes by commas first
    separate_val = re.split(r'[,( ]\s*', post_code.replace("- ", "-").replace("– ","–")) 
    for val in separate_val:

        #the within each value we check if we have a range
        if ((val.find('–')>0) or (val.find('-')>0)) :
            pc_range = re.split(r'[-–]\s*', val)
#             print(pc_range)
            #if we do have a range then we create all the post codes in that range and append them to the list
            try:
                for pc in range(int(pc_range[0]),int(pc_range[1])+1):
                #we append the post codes as strings and if need be we add leading zeros to make the string 5 characters long
                    post_code_list.append(str(pc).zfill(5))
            except:
                pass
                    
        else:
            try:
                int(val)
                post_code_list.append(val)
            except:
                pass
    return post_code_list

def createPostCode2StateMap():
    
    
    #first we create a list of all the post codes and the cities from DBpedia
    endpoint_url = "http://dbpedia.org/sparql"

    sparqlw = SPARQLWrapper(endpoint_url)
    sparqlw.setReturnFormat(JSON)

    query = """
    SELECT ?city ?state str(?cityName) ?iso2StateCode ?postCode
    WHERE {
        ?state  dct:subject dbc:States_of_the_United_States;
                dbo:postalCode ?iso2StateCode.
        ?city   a dbo:City;
                dbo:subdivision ?state;
                rdfs:label ?cityName;
                dbo:postalCode ?postCode.

    FILTER (?iso2StateCode != "").
    FILTER langMatches( lang(?cityName), "en" )
    }
    """
    sparqlw.setQuery(query)
    results = sparqlw.query().convert()
    
    #next we create a temporary dictionary with the post code string and the state code. However the post code in this dictionary is sometimes a range or a combination of post codes and ranges
    temp_post_dict = dict()
    for i in results['results']['bindings']:
        temp_post_dict[i['postCode']['value']] = i['iso2StateCode']['value']
    
    # Finally we reprocess the temp_post_dict post codes so we have extract all the post codes from the ranges and still map them to the same state
    postcode_to_state = dict()
    for post_code in temp_post_dict.keys():
        for pc in convertPostCodeStringToPostCodes(post_code):
            postcode_to_state[pc] = temp_post_dict[post_code]
    
    
    # next we create a dictionary with cities and states        
    city_to_state = dict()
    for row in results['results']['bindings']:
        city_to_state[row['callret-2']['value']] = row['iso2StateCode']['value']
    
    return postcode_to_state, city_to_state

def findStateByPostCodeCity(postcode, city, postcode_to_state, city_to_state):
    """
    A function used to map a given post code or city in the US to the respective state. Exact matches are expected instead of lexical similarity
    ...

    Attributes
    ----------
    postcode : str
        the post code to be mapped
    city : str
        the city to be mapped. Unlike the post code the city can be matched if it exists as a substring in the dictionary
    """
    
    try:
        return postcode_to_state[postcode]
    except:
        try:
            lst = [value for key, value in city_to_state.items() if city.lower() in key.lower()]
            return max(set(lst), key=lst.count)
        except:
            pass
            

postcode_to_state, city_to_state = createPostCode2StateMap()

solution.df['state_code'] = solution.df.apply(lambda x: x.state if len(x.state)==2 else findStateByPostCodeCity(x.postcode,x.city, postcode_to_state, city_to_state),axis=1)

In [7]:
solution.df.head()

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,restaurant_name,state_code
0,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Any Brick Oven Pizza,NaN,NaN,Small,Bertucci's___1,None
1,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Bbq Chicken Pizza,10.99,USD,BBQ chicken marinated in a smoky hickory BBQ s...,Bertucci's___1,None
2,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Buffalo Chicken Flatbread Pizza,8.49,USD,Grilled chicken tossed with buffalo sauce. Top...,Bertucci's___1,None
3,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Cheese Party Pizza,5.00,USD,NaN,Bertucci's___1,None
4,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Cheese Pizza,10.29,USD,NaN,Bertucci's___1,None


## 2.3 Addresses
We need to add one more column to the dataframe by concatenating the address column with the state column in order to generate unique values to use for the generation of the address class URI

In [8]:
solution.df['address_id'] = solution.df.apply(lambda x: x.state+'_'+x.address,axis=1)
solution.df.head()

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,restaurant_name,state_code,address_id
0,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Any Brick Oven Pizza,NaN,NaN,Small,Bertucci's___1,None,Htfd_2929 Berlin Tpke
1,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Bbq Chicken Pizza,10.99,USD,BBQ chicken marinated in a smoky hickory BBQ s...,Bertucci's___1,None,Htfd_2929 Berlin Tpke
2,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Buffalo Chicken Flatbread Pizza,8.49,USD,Grilled chicken tossed with buffalo sauce. Top...,Bertucci's___1,None,Htfd_2929 Berlin Tpke
3,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Cheese Party Pizza,5.00,USD,NaN,Bertucci's___1,None,Htfd_2929 Berlin Tpke
4,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Cheese Pizza,10.29,USD,NaN,Bertucci's___1,None,Htfd_2929 Berlin Tpke


## 2.4 Pizza
We need to add one more column to the dataframe by concatenating the menu item and the restaurant name in order to create a unique URI for pizzas that are served at different restaurants (i.e. if a pizza is served at 2 different restaurants has the same name we need to create to instances instead of one)

In [9]:
solution.df['pizza_name'] = solution.df.apply(lambda x: x.restaurant_name+'_'+str(x['menu item']),axis=1)
solution.df.head()

,name,address,city,country,postcode,state,categories,menu item,item value,currency,item description,restaurant_name,state_code,address_id,pizza_name
0,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Any Brick Oven Pizza,NaN,NaN,Small,Bertucci's___1,None,Htfd_2929 Berlin Tpke,Bertucci's___1_Any Brick Oven Pizza
1,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Bbq Chicken Pizza,10.99,USD,BBQ chicken marinated in a smoky hickory BBQ s...,Bertucci's___1,None,Htfd_2929 Berlin Tpke,Bertucci's___1_Bbq Chicken Pizza
2,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Buffalo Chicken Flatbread Pizza,8.49,USD,Grilled chicken tossed with buffalo sauce. Top...,Bertucci's___1,None,Htfd_2929 Berlin Tpke,Bertucci's___1_Buffalo Chicken Flatbread Pizza
3,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Cheese Party Pizza,5.00,USD,NaN,Bertucci's___1,None,Htfd_2929 Berlin Tpke,Bertucci's___1_Cheese Party Pizza
4,Bertucci's,2929 Berlin Tpke,Newington,US,6111.0,Htfd,"Restaurants,Italian Restaurants,Pizza",Cheese Pizza,10.29,USD,NaN,Bertucci's___1,None,Htfd_2929 Berlin Tpke,Bertucci's___1_Cheese Pizza


# CREATE TRIPLES

In [10]:
def getExternalKGURI(name):
    '''
    Approximate solution: We get the entity with highest lexical similarity
    The use of context may be necessary in some cases        
    '''
    
    dbpedia = DBpediaLookup()
    entities = dbpedia.getKGEntities(name, 5)
    #print("Entities from DBPedia:")
    current_sim = -1
    current_uri=''
    for ent in entities:           
        isub_score = isub(name, ent.label) 
        if current_sim < isub_score:
            current_uri = ent.ident
            current_sim = isub_score

#     print(current_uri)
    return current_uri

def loadStateISO2UrisFromDBPedia(classStringToURI):
    endpoint_url = "http://dbpedia.org/sparql"

    sparqlw = SPARQLWrapper(endpoint_url)
    sparqlw.setReturnFormat(JSON)
    
    #Create the sparql query to get the URIs for the states from the iso2 state code
    state_iso2_query = """
    SELECT ?state ?iso2StateCode
    WHERE {
        ?state  dct:subject dbc:States_of_the_United_States;
                dbo:postalCode ?iso2StateCode.
    FILTER (?iso2StateCode != "").
    }
    """
    
    sparqlw.setQuery(state_iso2_query)
    state_results = sparqlw.query().convert()

    # create a new key for the states and...
    classStringToURI['state_code'] = dict()

    # the the code to URI mappings
    for state in state_results['results']['bindings']:
        classStringToURI['state_code'][state['iso2StateCode']['value'].lower()] = state['state']['value']

def mappingToCreateTypeTriple(df, subject_column, class_type, use_external_uri, classStringToURI):
    
    # First we create a new key for the class with a value of another dictionary
    if subject_column != 'state_code':
        classStringToURI[subject_column] = dict()

    # Then we iterate through the rows in the subject column and either reuse an existing URI of contruct it from scratch
    for subject in df[subject_column]:
        
        #We use the subject_column value to create the fresh URI if this if the first time we see that value. 
        # If we've seen the value before we do not do anything since we've already added the tripple to the graph
        #################################### CHECK FOR NULL VALUES
        try:
            if subject.lower() not in classStringToURI[subject_column]:
                if use_external_uri:
                    entity_uri =  getExternalKGURI(subject.lower())
                else:
                    entity_uri = solution.zdetor_ns_str + subject.replace(" ", "_").replace("'","_").replace("&","_").replace("|","_")
                classStringToURI[subject_column][subject.lower()] = entity_uri
            else:
                entity_uri = classStringToURI[subject_column][subject.lower()]
                
            #Add the tripple to the KG
            solution.g.add((URIRef(entity_uri), RDF.type, class_type))
        except:
            pass

        
def mappingToCreateLiteralTriple(df, subject_column, object_column, predicate, datatype, classStringToURI):

    for subject, lit_value in zip(df[subject_column], df[object_column]):

        # check if the value is empty and if it is do not create the litteral value
        if solution.is_nan(lit_value) or lit_value==None or lit_value=="":
            pass

        else:
            try:
                #Uri as already created
                entity_uri=classStringToURI[subject_column][subject.lower()]

                #Literal
                lit = Literal(lit_value, datatype=datatype)

                #New triple
                solution.g.add((URIRef(entity_uri), predicate, lit))
            except:
                pass


def mappingToCreateObjectTriple(df, subject_column, object_column, predicate, classStringToURI):

    for subject, object in zip(df[subject_column], df[object_column]):
#         print(subject, object)

        if solution.is_nan(object) or object==None or object=="":
            pass

        else:
            #Uri as already created
            subject_uri=classStringToURI[subject_column][subject.lower()]
            object_uri=classStringToURI[object_column][object.lower()]

            #New triple
            solution.g.add((URIRef(subject_uri), predicate, URIRef(object_uri)))

## Create Classes

In [11]:
# print(solution.g.serialize(format="turtle").decode("utf-8"))
# solution.classStringToURI
# loadStateISO2UrisFromDBPedia(solution.classStringToURI)

In [12]:
# def CovertCSVToRDF(solution.df):
# this is an empty dictionary of the format {'class':{'string': uri}}

print('CREATING CLASSES:')

if 'restaurant_name' in solution.df:
    mappingToCreateTypeTriple(solution.df,'restaurant_name',solution.zdetor.Restaurant, False, solution.classStringToURI)
print('Restaurants complete')

solution.g.bind("dpo", Namespace("http://dbpedia.org/resource/"))
if 'city' in solution.df:
    mappingToCreateTypeTriple(solution.df,'city',solution.zdetor.City, True, solution.classStringToURI)
print('Cities complete')
    
if 'country' in solution.df:
    mappingToCreateTypeTriple(solution.df,'country',solution.zdetor.Country, True, solution.classStringToURI)
print('Countries complete')

loadStateISO2UrisFromDBPedia(solution.classStringToURI)
if 'state_code' in solution.df:
    mappingToCreateTypeTriple(solution.df, 'state_code',solution.zdetor.State, True, solution.classStringToURI)
print('States complete')
    
if 'address_id' in solution.df:
    mappingToCreateTypeTriple(solution.df,'address_id',solution.zdetor.Address, False, solution.classStringToURI)
print('Addresses complete')

if 'pizza_name' in solution.df:
    mappingToCreateTypeTriple(solution.df,'pizza_name',solution.zdetor.Pizza, False, solution.classStringToURI)
    mappingToCreateTypeTriple(solution.df,'pizza_name',solution.zdetor.MenuItem, False, solution.classStringToURI)
print('Pizzas complete')

if 'currency' in solution.df:
    mappingToCreateTypeTriple(solution.df,'currency',solution.zdetor.Currency, False, solution.classStringToURI)
print('Currency complete')

CREATING CLASSES:
Restaurants complete
Cities complete
Countries complete
States complete
Addresses complete
Pizzas complete
Currency complete


## Create Object Properties

In [13]:
if 'name' in solution.df:
    mappingToCreateObjectTriple(solution.df, 'restaurant_name','address_id',solution.zdetor.hasAddress, solution.classStringToURI)
    if 'pizza_name' in solution.df:
        mappingToCreateObjectTriple(solution.df, 'restaurant_name','pizza_name',solution.zdetor.hasMenuItem, solution.classStringToURI)

if 'address_id' in solution.df:
    mappingToCreateObjectTriple(solution.df, 'address_id','city',solution.zdetor.hasCity, solution.classStringToURI)
    mappingToCreateObjectTriple(solution.df, 'address_id','state_code',solution.zdetor.hasState, solution.classStringToURI)
    mappingToCreateObjectTriple(solution.df, 'address_id','country',solution.zdetor.hasCountry, solution.classStringToURI)
    
if 'pizza_name' in solution.df:
     mappingToCreateObjectTriple(solution.df, 'pizza_name','currency',solution.zdetor.hasCurrency, solution.classStringToURI)

## Create Literals

In [14]:
if 'name' in solution.df:
    mappingToCreateLiteralTriple(solution.df, 'restaurant_name','name',solution.zdetor.name, XSD.string, solution.classStringToURI)
    
if 'city' in solution.df:
    mappingToCreateLiteralTriple(solution.df, 'city','city',solution.zdetor.name, XSD.string, solution.classStringToURI)
print('Cities complete')
    
if 'country' in solution.df:
    mappingToCreateLiteralTriple(solution.df, 'country','country',solution.zdetor.name, XSD.string, solution.classStringToURI)
print('Countries complete')

if 'state_code' in solution.df:
    mappingToCreateLiteralTriple(solution.df, 'state_code','state_code',solution.zdetor.name, XSD.string, solution.classStringToURI)
print('States complete')
    
if 'address_id' in solution.df:
    mappingToCreateLiteralTriple(solution.df, 'address_id','address',solution.zdetor.addressLine, XSD.string, solution.classStringToURI)
    mappingToCreateLiteralTriple(solution.df, 'address_id','postcode',solution.zdetor.postCode, XSD.string, solution.classStringToURI)
    
if 'pizza_name' in solution.df:
    mappingToCreateLiteralTriple(solution.df, 'pizza_name','item value',solution.zdetor.price, XSD.float, solution.classStringToURI)
    mappingToCreateLiteralTriple(solution.df, 'pizza_name','currency',solution.zdetor.currency, XSD.string, solution.classStringToURI)
    mappingToCreateLiteralTriple(solution.df, 'pizza_name','menu item',solution.zdetor.name, XSD.string, solution.classStringToURI)
    mappingToCreateLiteralTriple(solution.df, 'pizza_name','item description',solution.zdetor.description, XSD.string, solution.classStringToURI)

Cities complete
Countries complete
States complete


# Extract Knowledge from String Columns

In [15]:
# concatenate all the values from the categories column to create the string to perform the NLP on

def createListOfFrequentTerms(df, column, max_ngram_size = 2, numOfKeywords = 40):
    concat_string = ''

    for val in df[column]:
        if solution.is_nan(val) or val==None or val=="":
            pass
        else:
            concat_string = concat_string + ', ' + str(val)

    kw_extractor = yake.KeywordExtractor()

    # text = """spaCy is an open-source software library for advanced natural language processing, written in the programming languages Python and Cython. The library is published under the MIT license and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
    text = concat_string
    language = "en"
    deduplication_threshold = 0.9
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)

    sorted_list = []
    for kw in keywords:
        sorted_list.append(kw[0])
        print(kw)
    return sorted_list

# frequent_restaurant_categories = createListOfFrequentTerms(solution.df, 'item description', numOfKeywords = 10)

# frequent_pizza_classes = createListOfFrequentTerms(solution.df, 'menu item', numOfKeywords = 20)

# frequent_toppings = createListOfFrequentTerms(solution.df, 'item description', max_ngram_size=2, numOfKeywords = 40)

In [16]:
#Method from owlready
from owlready2 import *

def getClasses(onto):        
        return onto.classes()
    
def getOntoClassesByTerm(urionto, prefix, parent_class):

    # load the ontology
    onto = get_ontology(urionto).load()
    
    # get all ontology classes
    entities = list(getClasses(onto))

    
    #create a dictionary with subclasses of the parent_class. Assuming the name of the parent class is there as a suffix in the subclass
    classes = dict()
    for entity in entities:
        #expectein the name of the parent class to appear in the subclass name but NOT immediately after the prefix
        if str(entity).find(parent_class)>len(prefix)+1:
            classes[str(entity).replace(prefix+".","").replace(parent_class,"").lower()] = str(entity).replace('zdetor.',solution.zdetor_ns_str)
    return classes

categories = getOntoClassesByTerm('./input_files/zdetor.owl','zdetor','Restaurant')
pizzas = getOntoClassesByTerm('./input_files/zdetor.owl','zdetor','Pizza')
toppings = getOntoClassesByTerm('./input_files/zdetor.owl','zdetor','Topping')

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [17]:
solution.classStringToURI['topping'] = dict()
for topping in toppings:

    entity_uri = solution.zdetor_ns_str+topping
    solution.classStringToURI['topping'][topping] = entity_uri
    solution.g.add((URIRef(entity_uri), RDF.type, URIRef(toppings[topping])))

In [18]:
def mappingToCreateObjectProperty(df, subject_column, object_column, object_dict, classStringToURI, predicate = RDF.type):

    for subject, object in zip(df[subject_column], df[object_column]):

        if solution.is_nan(object) or object==None or object=="":
            pass

        else:
            separate_val = set(re.split(r'[,( ]\s*', object.lower()))
#             print(subject, object, separate_val)
            for val in separate_val:
                try:
#                     print(val)
                    subject_uri=classStringToURI[subject_column][subject.lower()]
                    solution.g.add((URIRef(subject_uri), predicate, URIRef(object_dict[val])))
                except:
                    pass
                
def mappingToCreatePizzaToppings(df, subject_column, object_column, object_dict, classStringToURI, predicate = RDF.type):

    for subject, object in zip(df[subject_column], df[object_column]):

        if solution.is_nan(object) or object==None or object=="":
            pass

        else:
            separate_val = set(re.split(r'[.,( ]\s*', object.lower()))
#             print(subject, object, separate_val)
            for val in separate_val:
                try:
#                     print(val)
                    subject_uri=classStringToURI[subject_column][subject.lower()]
                    solution.g.add((URIRef(subject_uri), predicate, URIRef(object_dict[val])))
                except:
                    pass

In [19]:
mappingToCreateObjectProperty(solution.df,'restaurant_name','categories',categories,solution.classStringToURI)
mappingToCreateObjectProperty(solution.df,'pizza_name','menu item',pizzas,solution.classStringToURI)
# mappingToCreateObjectProperty(solution.df,'pizza_name','item description',toppings,solution.classStringToURI,solution.zdetor.hasTopping)

mappingToCreatePizzaToppings(solution.df,'pizza_name','item description',solution.classStringToURI['topping'],solution.classStringToURI,solution.zdetor.hasTopping)

In [23]:
# print(solution.g.serialize(format="turtle").decode("utf-8"))
# solution.classStringToURI

In [ ]:
def saveGraph(graph, file_output):

    graph.serialize(destination=file_output, format='ttl')
    print("Triples including ontology: '" + str(len(graph)) + "'.")

def performReasoning(graph, ontology_file):

    print("Triples including ontology: '" + str(len(graph)) + "'.")
    
    #We should load the ontology first
    graph.load(ontology_file,  format='ttl') #e.g., format=ttl


    #We apply reasoning and expand the graph with new triples 
    owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=False, datatype_axioms=False).expand(graph)

    print("Triples after OWL 2 RL reasoning: '" + str(len(graph)) + "'.")

# 3 REASONING

In [ ]:
# print(solution.g.serialize(format="turtle").decode("utf-8"))

In [ ]:
saveGraph(solution.g, './input_files/cw-data.ttl')

In [1]:
'''
Created on 01 April 2021
@author: zacharias.detorakis@city.ac.uk

'''

# Import Libraries
import sys
sys.path.append('./lib/')

import pandas as pd
import yake
import nltk

# concatenate all the values from the categories column to create the string to perform the NLP on
def is_nan(x):
        return (x != x)

def createListOfFrequentTerms(df, column, max_ngram_size = 2, numOfKeywords = 40):
    concat_string = ''

    for val in df[column]:
        if is_nan(val) or val==None or val=="":
            pass
        else:
            concat_string = concat_string + ', ' + str(val)

    kw_extractor = yake.KeywordExtractor()

    # text = """spaCy is an open-source software library for advanced natural language processing, written in the programming languages Python and Cython. The library is published under the MIT license and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
    text = concat_string
    language = "en"
    deduplication_threshold = 0.9
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)

    sorted_list = []
    for kw in keywords:
        sorted_list.append(kw[0])
        print(kw)
#     return sorted_list
    return concat_string

input_csv = "./input_files/INM713_coursework_data_pizza_8358_1_reduced.csv"
# input_csv = "./input_files/INM713_coursework_data_pizza_8358_1_reduced - small.csv"


df = pd.read_csv(filepath_or_buffer = input_csv, sep=',', quotechar='"',escapechar="\\")

frequent_restaurant_categories = createListOfFrequentTerms(df, 'item description', numOfKeywords = 10)

# frequent_pizza_classes = createListOfFrequentTerms(solution.df, 'menu item', numOfKeywords = 20)

# frequent_toppings = createListOfFrequentTerms(solution.df, 'item description', max_ngram_size=2, numOfKeywords = 40)

C:\Users\zacharias.detorakis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\zacharias.detorakis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` he

('mozzarella cheese', 2.460707156597209e-05)
('fresh mozzarella', 5.9493536942988445e-05)
('mozzarella', 9.005796818250687e-05)
('cheese', 0.00012257104454523586)
('tomato sauce', 0.00013391404142679226)
('fresh', 0.00020205086203522446)
('fresh garlic', 0.00021545366628884633)
('sauce', 0.00022454942569216028)
('fresh basil', 0.00025266220774878076)
('grilled chicken', 0.00025771056486215935)


In [2]:
text = frequent_restaurant_categories
# text

In [3]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from pattern.text.en import singularize

# text = "Nick likes to play football, however he is not too fond of tennis."
words = nltk.word_tokenize(text)
new_words= [word for word in words if word.isalnum()]

tokens_without_sw = [word for word in new_words if not word in stopwords.words()]

# singles = [singularize(plural) for plural in tokens_without_sw]
print(tokens_without_sw)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zacharias.detorakis\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package stopwords is already up-to-date!


['Olives', 'onions', 'capers', 'tomatoes', 'Choose', 'pizza', 'Canadian', 'bacon', 'Choose', 'toppings', 'make', 'Seven', 'layer', 'bean', 'dip', '1', 'g', '1', 'g', 'pepperoni', 'mushroom', 'pepperoni', 'mushroom', 'BBQ', 'Sauce', 'Mozzarella', 'Cheese', 'diced', 'Chicken', 'Breast', 'Onions', 'Pepperoni', 'marinara', 'garlic', 'seasoning', 'parmesan', 'cheese', 'Our', 'original', 'round', 'pizza', 'topped', 'pepperoni', 'sausage', 'Our', 'original', 'round', 'pizza', 'topped', 'pepperoni', 'Italian', 'sausage', 'mushroom', 'onion', 'green', 'Broccoli', 'spinach', 'peppers', 'mushrooms', 'Sausage', 'pepperoni', 'meatballs', 'peppers', 'onions', 'black', 'olives', 'mushrooms', 'Fresh', 'mozzarella', 'fresh', 'parmesan', 'fresh', 'garlic', 'Italian', 'pepperoni', 'hot', 'italian', 'sausage', 'fresh', 'ground', 'western', 'beef', 'cob', 'smoked', 'bacon', 'marinated', 'chicken', 'breast', 'new', 'england', 'baked', 'anchovy', 'filets', 'fresh', 'plum', 'tomatoes', 'freshly', 'cut', 'broc

In [5]:
kw_extractor = yake.KeywordExtractor()

# text = """spaCy is an open-source software library for advanced natural language processing, written in the programming languages Python and Cython. The library is published under the MIT license and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
text = ' '.join(tokens_without_sw)
language = "en"
deduplication_threshold = 0.9
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=2, dedupLim=0.9, top=40, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

sorted_list = []
for kw in keywords:
    sorted_list.append(kw[0])
    print(kw)
# sorted_list

('mozzarella cheese', 2.885310645364898e-07)
('fresh mozzarella', 5.691008062270546e-07)
('sauce mozzarella', 7.047959125173455e-07)
('tomato sauce', 1.251814908229208e-06)
('cheese pepperoni', 1.6162285567141825e-06)
('fresh garlic', 1.7148384825677237e-06)
('grilled chicken', 1.8756831657670691e-06)
('mozzarella fresh', 1.8870184627528653e-06)
('fresh basil', 1.981108447968706e-06)
('cheese fresh', 2.0045554310191117e-06)
('cheese mozzarella', 2.1867617522765543e-06)
('bbq sauce', 2.396313680483577e-06)
('cheese topped', 2.469483806558005e-06)
('garlic mozzarella', 2.4725826285425254e-06)
('fresh tomato', 2.6349991613148235e-06)
('cheese sauce', 2.920372047226696e-06)
('pizza sauce', 3.0583159475494283e-06)
('sauce fresh', 3.092799211060654e-06)
('mozzarella tomato', 3.124461694392953e-06)
('pepperoni sausage', 3.1550559536699496e-06)
('ricotta mozzarella', 3.2307779197878614e-06)
('fresh tomatoes', 3.2530890535683776e-06)
('onions fresh', 3.3365404653967247e-06)
('chicken mozzarella